In [68]:
import requests
import pandas as pd
from tqdm import tqdm
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [69]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + '1'
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
driver.get(url)
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [70]:
# View 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

body = driver.find_element(By.TAG_NAME, 'body')
for _ in range(16):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    # 웹스크롤 실행되는 동안 다른 명령x, 끝날 때까지 기다리기(30번*1초) 

In [71]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [72]:
len(driver.page_source), len(res.text)

(298164, 292114)

- Step 3

In [73]:
tr = trs[0]
rank = int(tr.select_one('.rank').get_text().strip())
rank

1

In [74]:
category= tr.select_one('.category').get_text().strip()[1:-1]
category

'음악/댄스/가수'

In [75]:
# channel = tr.find('h1 > a').get_text().strip()
channel = tr.select_one('h1 > a').get_text().strip()
channel

'BLACKPINK'

In [76]:
subs = tr.select_one('.subscriber_cnt').get_text().strip()
subs

'8800만'

In [77]:
view = tr.select_one('.view_cnt').get_text().strip()    # int로 바꿔야함
video = (tr.select_one('.video_cnt').get_text()[:-1])
view, video

('309억5913만', '514')

In [78]:
# 문자열 s -> int(s)로 바꿔주는 함수
def convert(s):
    s= s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [79]:
convert('333억8260')

3338260

- Step4

In [80]:
lines = []
for tr in trs:
    rank = int(tr.select_one('.rank').get_text().strip())
    category= tr.select_one('.category').get_text().strip()[1:-1]
    channel = tr.select_one('h1 > a').get_text().strip()
    subs = convert(tr.select_one('.subscriber_cnt').get_text().strip())
    view = convert(tr.select_one('.view_cnt').get_text().strip())
    video = convert((tr.select_one('.video_cnt').get_text()[:-1]))
    lines.append({
        '순위': rank, '카테고리':category, '채널명':channel, 
        '구독자수':subs, '조회수':view, '비디오':video
        })

In [81]:
df=pd.DataFrame(lines)
df.tail(7)

,순위,카테고리,채널명,구독자수,조회수,비디오
93,94,음악/댄스/가수,ASTRO 아스트로,5130000,639100000,646
94,95,TV/방송,MBCdrama,4940000,5779930000,60148
95,96,키즈/어린이,뽀로로(Pororo),4910000,6878710000,4182
96,97,음식/요리/레시피,떵개떵,4890000,2990060000,4924
97,98,키즈/어린이,ToyMart TV,4860000,1994040000,695
98,99,애완/반려동물,SBS TV동물농장x애니멀봐,4840000,4654750000,4200
99,101,미분류,매일맛나 delicious day,4780000,488910000,328


- 두 번째 페이지

In [82]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(2)
driver.get(url)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('.aos-init')
len(trs)

100

In [83]:
driver.close()

In [84]:
# 1~ 모든 p
lines = []
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver_win32/chromedriver.exe')
for page in tqdm(range(1, 11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    
    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category= tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('h1 > a').get_text().strip()
        subs = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view = convert(tr.select_one('.view_cnt').get_text().strip())
        video = convert((tr.select_one('.video_cnt').get_text()[:-1]))
        lines.append({
            '순위': rank, '카테고리':category, '채널명':channel, 
            '구독자수':subs, '조회수':view, '비디오':video
            })
        
driver.close()

100%|██████████| 10/10 [00:37<00:00,  3.80s/it]


- Step 7

In [85]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [86]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
995,1003,미분류,빨간내복야코,730000,508340000,246
996,997,키즈/어린이,코코몽 COCOMONG TV - Cartoon & Song For Kids,730000,951630000,1420
997,996,취미/라이프,DanoTV,730000,158420000,642
998,1001,미분류,김지윤의 지식Play,730000,59060000,186
999,1000,미분류,성창경TV,730000,1036110000,16348


In [87]:
df.to_csv('data/20230425_유튜브랭킹.csv', index=False)